In [89]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

In [32]:
# Load in data, form is ID, mass, size, WHIM sizes
d2048 = np.loadtxt('./2048z3/WHIM_data.txt')

In [90]:
# Define useful tools:
TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

In [91]:
# First make a basic scatter plot
p = figure(tools=TOOLS)
p.scatter(x=np.transpose(np.log10([d2048[:,1]]*26)).flatten(), y=d2048[:,3::].flatten())
show(p)

In [104]:
# First plot histogram of mass distribution

# Take log of halo masses:
log_hm = np.log10(d2048[:,1])
# How many bins do we want?
nbins = 10

# Get bin positions
hist, edges = np.histogram(log_hm, density=False, bins=nbins)

p1 = figure(title='',tools="save", background_fill_color="#E8DDCB")
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color="#036564", line_color="#033649")
show(p1)

In [93]:
# Now make a box plot

for i in range(len(d2048z3)):
    
df = pd.DataFrame(dict(score=yy, group=g))


# Get position for lines (weighted bin position)
pos_bin = [edges[i+1]-edges[i] for i in range(nbins)]
pos_line = [log_hm[bin_ind[i]:bin_ind[i+1]].sum()/(bin_ind[i+1]-bin_ind[i]) for i in range(nbins)]

cats = list("abcdef")
yy = np.random.randn(2000)
g = np.random.choice(cats, 2000)
for i, l in enumerate(cats):
    yy[g == l] += i // 2
df = pd.DataFrame(dict(score=yy, group=g))

# find the quartiles and IQR for each category
groups = df.groupby('group')
q1 = groups.quantile(q=0.25)
q2 = groups.quantile(q=0.5)
q3 = groups.quantile(q=0.75)
# Let's use this definition rather than the standard one.
upper = 0.95
lower = 0.05

# find the outliers for each category
def outliers(group):
    cat = group.name
    return group[(group.score > upper.loc[cat]['score']) | (group.score < lower.loc[cat]['score'])]['score']
out = groups.apply(outliers).dropna()

# prepare outlier data for plotting, we need coordinates for every outlier.
if not out.empty:
    outx = []
    outy = []
    for cat in cats:
        # only add outliers if they exist
        if not out.loc[cat].empty:
            for value in out[cat]:
                outx.append(cat)
                outy.append(value)

p2 = figure(tools="save", background_fill_color="#EFE8E2", title="", x_range=cats)

# if no outliers, shrink lengths of stems to be no longer than the minimums or maximums
qmin = groups.quantile(q=0.00)
qmax = groups.quantile(q=1.00)
upper.score = [min([x,y]) for (x,y) in zip(list(qmax.loc[:,'score']),upper.score)]
lower.score = [max([x,y]) for (x,y) in zip(list(qmin.loc[:,'score']),lower.score)]

# stems
p.segment(cats, upper.score, cats, q3.score, line_color="black")
p.segment(cats, lower.score, cats, q1.score, line_color="black")

# boxes
p.vbar(cats, 0.7, q2.score, q3.score, fill_color="#E08E79", line_color="black")
p.vbar(cats, 0.7, q1.score, q2.score, fill_color="#3B8686", line_color="black")

# whiskers (almost-0 height rects simpler than segments)
p.rect(cats, lower.score, 0.2, 0.01, line_color="black")
p.rect(cats, upper.score, 0.2, 0.01, line_color="black")

# outliers
if not out.empty:
    p.circle(outx, outy, size=6, color="#F38630", fill_alpha=0.6)

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = "white"
p.grid.grid_line_width = 2
p.xaxis.major_label_text_font_size="12pt"

show(p)

AttributeError: 'float' object has no attribute 'loc'